In [1]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import stats
import matplotlib.pyplot as plt

from compute_tempo_aist import *
from aist_pos1s_script import *
# coco={    
# 0: "nose", 1: "left_eye", 2: "right_eye", 3: "left_ear",4: "right_ear",5: "left_shoulder",
# 6: "right_shoulder",7: "left_elbow",8: "right_elbow",9: "left_wrist",10: "right_wrist",
# 11: "left_hip",12: "right_hip",13: "left_knee",14: "right_knee",15: "left_ankle",16: "right_ankle",}  

# # cp ./video/gKR_sBM_c01_d28_mKR3_ch03.mp4 ./


# main_dir = "/itf-fi-ml/home/sagardu/aist_tempo_est/optimization/saved_results/tempo_case_1"

# Define the folder structure
# directories = [
#     "pos/tempo_70_145/foot","pos/tempo_70_145/hand","pos/tempo_70_145/score/foot","pos/tempo_70_145/score/hand",
#     "vel/tempo_70_145/foot","vel/tempo_70_145/hand","vel/tempo_70_145/score/foot","vel/tempo_70_145/score/hand",
#     "stats_pos/tempo_70_145/uni","stats_pos/tempo_70_145/bi",
#     "stats_vel/tempo_70_145/uni","stats_vel/tempo_70_145/bi",
#     "stats_posvel/tempo_70_145/uni","stats_posvel/tempo_70_145/bi"
# ]

# for dir_path in directories:
#     full_path = os.path.join(main_dir, dir_path)
#     os.makedirs(full_path, exist_ok=True)
# print("All directories and subdirectories have been created successfully!")


#### Window Size parameter optimization

In [ ]:
marker_dict = {9: "left_wrist", 10: "right_wrist", 
                15: "left_ankle", 16: "right_ankle", 
                }   # 11: "left_hip",12: "right_hip"

# config1 = {"sub_dir": ["hand"], "mode": ["zero_uni", "zero_bi"], "markerA_id": [9, 10], "a": 70, "b": 145}
# config2 = {"sub_dir": ["foot"], "mode": ["zero_uni", "zero_bi"], "markerA_id": [15, 16], "a": 70, "b": 145}
# config3 = {"sub_dir": ["hand"], "mode": ["zero_uni", "zero_bi"], "markerA_id": [9, 10], "a": 60, "b": 180}
# config4 = {"sub_dir": ["foot"], "mode": ["zero_uni", "zero_bi"], "markerA_id": [15, 16], "a": 60, "b": 180}

config1 = {"sub_dir": ["hand"], "mode": ["zero_uni", "zero_bi"], "markerA_id": [9, 10], "a": 70, "b": 145,
           "win_size_list": np.arange(1,14).tolist()}      # 4 beat window

config2 = {"sub_dir": ["foot"], "mode": ["zero_uni", "zero_bi"], "markerA_id": [15, 16], "a": 70, "b": 145,
           "win_size_list": np.arange(1,14).tolist()}      # 4 beat window



case = "win_case_2"
configs = [config1, config2]

total_iterations = sum(
    len(cfg["sub_dir"]) * len(cfg["mode"]) * len(cfg["markerA_id"]) * len(cfg["win_size_list"]) * 4
    for cfg in configs
)

with tqdm(total=total_iterations, desc="Processing Configurations") as pbar:
    for cfg in configs:
        a = cfg["a"]
        b = cfg["b"]
        for sub_dir in cfg["sub_dir"]:
            for mode in cfg["mode"]:
                for markerA_id in cfg["markerA_id"]:
                    for w_sec in cfg['win_size_list']:
                        for h_sec in [w_sec*0.25, w_sec*0.5, w_sec*0.75, w_sec*1]:
                            # h_sec = w_sec/4
                            # savepath = f"./saved_results/{case}/tempo_{a}_{b}"
                            csv_dir = f"./saved_results/{case}/vel/tempo_{a}_{b}/{sub_dir}"
                            csv_filename = f"{marker_dict[markerA_id]}_W{w_sec}H{h_sec}_{mode}_{a}_{b}.csv"
                            csv_file_path = os.path.join(csv_dir, csv_filename)
                            
                            aist_pos1s(a, b, mode, markerA_id, csv_file_path, w_sec, h_sec, vel_mode= "on")
                            pbar.update(1)

#### Tempo Parameterization

In [ ]:
def create_dir(tempo_dir):
    
    main_dir = "/itf-fi-ml/home/sagardu/aist_tempo_est/optimization/saved_results/tempo_cases"
    
    directories = [
        f"{tempo_dir}/pos/foot", f"{tempo_dir}/pos/hand", f"{tempo_dir}/pos/score/foot", f"{tempo_dir}/pos/score/hand",
        f"{tempo_dir}/vel/foot", f"{tempo_dir}/vel/hand", f"{tempo_dir}/vel/score/foot", f"{tempo_dir}/vel/score/hand",
        f"{tempo_dir}/stats_pos/uni", f"{tempo_dir}/stats_pos/bi",
        f"{tempo_dir}/stats_vel/uni", f"{tempo_dir}/stats_vel/bi",
        f"{tempo_dir}/stats_posvel/uni", f"{tempo_dir}/stats_posvel/bi"
    ]
    
    for dir_path in directories:
        full_path = os.path.join(main_dir, dir_path)
        os.makedirs(full_path, exist_ok=True)
    # print(f"Subdirectories created inside {tempo_dir}!")

In [ ]:

marker_dict = {9: "left_wrist", 10: "right_wrist", 
                15: "left_ankle", 16: "right_ankle", 
                }   # 11: "left_hip",12: "right_hip"

config1 = {"sub_dir": ["hand"], "mode": ["zero_uni", "zero_bi"], "markerA_id": [9, 10], 
           "a": 60, 
           "b": [100,110,120,130,140,150,160,170,180,190,200],
           "win_size": 5}      # 4 beat window

config2 = {"sub_dir": ["foot"], "mode": ["zero_uni", "zero_bi"], "markerA_id": [15, 16], 
           "a": 60, 
           "b": [100,110,120,130,140,150,160,170,180,190,200],
           "win_size": 5}      # 4 beat window


metric = "vel"
case = "tempo_cases"
configs = [config1, config2]

total_iterations = 0
for cfg in configs:
    total_iterations += len(cfg["sub_dir"]) * len(cfg["mode"]) * len(cfg["markerA_id"]) * len(cfg["b"])

with tqdm(total=total_iterations, desc="Overall progress") as pbar:
    for cfg in configs:
        a = cfg["a"]
        b = cfg["b"]
        w_sec = cfg["win_size"]
        h_sec = w_sec/2
        
        for b in cfg['b']:
            create_dir(f"tempo_{a}_{b}")
            for sub_dir in cfg["sub_dir"]:
                for mode in cfg["mode"]:
                    for markerA_id in cfg["markerA_id"]:

                        csv_dir = f"./saved_results/{case}/tempo_{a}_{b}/{metric}/{sub_dir}"
                        csv_filename = f"{marker_dict[markerA_id]}_W{w_sec}H{h_sec}_{mode}_{a}_{b}.csv"
                        csv_file_path = os.path.join(csv_dir, csv_filename)
                        
                        aist_pos1s(a, b, mode, markerA_id, csv_file_path, w_sec, h_sec, vel_mode= "on")
                        pbar.update(1)

In [ ]:
f_path = "../aist_dataset/aist_annotation/keypoints2d"
aist_filelist = os.listdir(f_path)


duration_list = []
for idx, filename in enumerate(tqdm(aist_filelist)):
    
    file_path = os.path.join(f_path, filename)
    with open(file_path, 'rb') as file:
            motion_data = pickle.load(file)

    markerA_x = motion_data["keypoints2d"][0, :, 9, 0]
    duration = int(len(markerA_x)/60)
    duration_list.append(duration)

print("min:", min(duration_list))
print("max:", max(duration_list))

duration_arr = np.array(duration_list)

In [ ]:
plt.hist(duration_arr[duration_arr<13], bins=10)
print(duration_arr[duration_arr<13].shape)

#### Skipped Files Analysis

In [ ]:
Skipped_files = ['gPO_sMM_cAll_d12_mPO2_ch07.pkl', 'gBR_sBM_cAll_d04_mBR2_ch05.pkl', 'gJS_sFM_cAll_d03_mJS1_ch02.pkl', 'gMH_sMM_cAll_d24_mMH3_ch10.pkl', 'gLH_sMM_cAll_d18_mLH5_ch10.pkl', 'gJS_sMM_cAll_d02_mJS3_ch03.pkl', 'gBR_sBM_cAll_d05_mBR1_ch10.pkl', 'gPO_sMM_cAll_d11_mPO2_ch04.pkl', 'gBR_sFM_cAll_d06_mBR5_ch21.pkl', 'gJB_sMM_cAll_d09_mJB5_ch10.pkl', 'gLO_sMM_cAll_d15_mLO4_ch09.pkl', 'gBR_sMM_cAll_d06_mBR1_ch07.pkl', 'gHO_sMM_cAll_d19_mHO2_ch02.pkl', 'gBR_sBM_cAll_d04_mBR3_ch08.pkl', 'gLO_sMM_cAll_d14_mLO5_ch06.pkl', 'gJS_sFM_cAll_d03_mJS5_ch13.pkl', 'gJS_sMM_cAll_d01_mJS3_ch03.pkl', 'gLH_sMM_cAll_d17_mLH3_ch05.pkl', 'gWA_sMM_cAll_d27_mWA0_ch07.pkl', 'gKR_sMM_cAll_d30_mKR1_ch08.pkl', 'gHO_sFM_cAll_d20_mHO1_ch09.pkl', 'gBR_sBM_cAll_d05_mBR4_ch07.pkl', 'gHO_sMM_cAll_d21_mHO5_ch10.pkl', 'gBR_sBM_cAll_d04_mBR1_ch07.pkl', 'gWA_sMM_cAll_d25_mWA4_ch02.pkl', 'gLH_sMM_cAll_d17_mLH5_ch06.pkl', 'gJS_sFM_cAll_d02_mJS1_ch02.pkl', 'gPO_sMM_cAll_d11_mPO3_ch05.pkl', 'gBR_sMM_cAll_d05_mBR2_ch05.pkl', 'gMH_sMM_cAll_d23_mMH2_ch06.pkl', 'gBR_sBM_cAll_d05_mBR1_ch08.pkl', 'gBR_sBM_cAll_d05_mBR1_ch07.pkl', 'gJB_sMM_cAll_d07_mJB4_ch02.pkl', 'gBR_sBM_cAll_d06_mBR3_ch07.pkl', 'gBR_sMM_cAll_d06_mBR3_ch08.pkl', 'gBR_sBM_cAll_d05_mBR0_ch10.pkl', 'gBR_sMM_cAll_d06_mBR4_ch09.pkl', 'gJB_sMM_cAll_d07_mJB5_ch03.pkl', 'gJS_sFM_cAll_d02_mJS1_ch11.pkl', 'gLH_sMM_cAll_d16_mLH0_ch01.pkl', 'gLO_sMM_cAll_d13_mLO1_ch02.pkl', 'gJS_sFM_cAll_d01_mJS1_ch02.pkl', 'gMH_sMM_cAll_d24_mMH1_ch08.pkl', 'gBR_sBM_cAll_d04_mBR2_ch10.pkl', 'gPO_sMM_cAll_d10_mPO3_ch02.pkl', 'gKR_sFM_cAll_d30_mKR0_ch15.pkl', 'gMH_sMM_cAll_d22_mMH1_ch02.pkl', 'gBR_sMM_cAll_d05_mBR1_ch04.pkl', 'gMH_sMM_cAll_d23_mMH1_ch05.pkl', 'gBR_sBM_cAll_d05_mBR0_ch07.pkl', 'gKR_sFM_cAll_d29_mKR5_ch13.pkl', 'gJB_sMM_cAll_d07_mJB3_ch01.pkl', 'gBR_sBM_cAll_d04_mBR0_ch10.pkl', 'gJS_sFM_cAll_d02_mJS5_ch10.pkl', 'gLO_sMM_cAll_d14_mLO4_ch05.pkl', 'gJS_sFM_cAll_d03_mJS4_ch12.pkl', 'gLH_sMM_cAll_d17_mLH0_ch04.pkl', 'gLH_sMM_cAll_d16_mLH3_ch03.pkl', 'gKR_sMM_cAll_d29_mKR0_ch04.pkl', 'gJS_sFM_cAll_d01_mJS0_ch01.pkl', 'gHO_sMM_cAll_d20_mHO2_ch04.pkl', 'gLO_sMM_cAll_d13_mLO5_ch03.pkl', 'gBR_sFM_cAll_d04_mBR5_ch06.pkl', 'gKR_sMM_cAll_d30_mKR2_ch09.pkl', 'gKR_sMM_cAll_d28_mKR2_ch03.pkl', 'gHO_sMM_cAll_d20_mHO4_ch06.pkl', 'gBR_sMM_cAll_d04_mBR4_ch02.pkl', 'gBR_sBM_cAll_d05_mBR1_ch09.pkl', 'gBR_sFM_cAll_d04_mBR4_ch05.pkl', 'gHO_sMM_cAll_d19_mHO3_ch03.pkl', 'gBR_sBM_cAll_d04_mBR3_ch05.pkl', 'gBR_sBM_cAll_d06_mBR5_ch08.pkl', 'gBR_sMM_cAll_d04_mBR5_ch03.pkl', 'gBR_sFM_cAll_d06_mBR2_ch16.pkl', 'gBR_sBM_cAll_d04_mBR3_ch07.pkl', 'gLH_sMM_cAll_d18_mLH1_ch07.pkl', 'gBR_sBM_cAll_d06_mBR5_ch05.pkl', 'gHO_sFM_cAll_d19_mHO3_ch04.pkl', 'gBR_sMM_cAll_d06_mBR5_ch10.pkl', 'gJS_sMM_cAll_d02_mJS0_ch04.pkl', 'gMH_sMM_cAll_d24_mMH2_ch09.pkl', 'gHO_sMM_cAll_d20_mHO3_ch05.pkl', 'gJS_sMM_cAll_d02_mJS1_ch02.pkl', 'gPO_sMM_cAll_d12_mPO3_ch08.pkl', 'gJS_sMM_cAll_d03_mJS5_ch05.pkl', 'gBR_sBM_cAll_d06_mBR2_ch10.pkl', 'gBR_sMM_cAll_d04_mBR3_ch01.pkl', 'gJB_sFM_cAll_d08_mJB0_ch08.pkl', 'gPO_sMM_cAll_d12_mPO5_ch09.pkl', 'gJB_sMM_cAll_d09_mJB1_ch08.pkl', 'gBR_sBM_cAll_d06_mBR3_ch08.pkl', 'gMH_sFM_cAll_d22_mMH1_ch02.pkl', 'gHO_sFM_cAll_d20_mHO0_ch08.pkl', 'gBR_sBM_cAll_d05_mBR0_ch08.pkl', 'gWA_sMM_cAll_d25_mWA5_ch03.pkl', 'gBR_sFM_cAll_d04_mBR1_ch02.pkl', 'gKR_sMM_cAll_d29_mKR5_ch06.pkl', 'gJS_sMM_cAll_d03_mJS0_ch01.pkl', 'gWA_sMM_cAll_d26_mWA5_ch06.pkl', 'gJB_sFM_cAll_d07_mJB3_ch07.pkl', 'gLO_sMM_cAll_d15_mLO5_ch10.pkl', 'gLH_sMM_cAll_d18_mLH3_ch09.pkl', 'gWA_sMM_cAll_d26_mWA3_ch05.pkl', 'gBR_sFM_cAll_d04_mBR4_ch07.pkl', 'gJB_sMM_cAll_d09_mJB0_ch07.pkl', 'gBR_sBM_cAll_d05_mBR5_ch08.pkl', 'gLO_sMM_cAll_d13_mLO0_ch01.pkl', 'gLO_sMM_cAll_d15_mLO0_ch07.pkl', 'gBR_sBM_cAll_d04_mBR0_ch08.pkl', 'gPO_sMM_cAll_d10_mPO5_ch03.pkl', 'gMH_sMM_cAll_d23_mMH0_ch04.pkl', 'gJS_sMM_cAll_d03_mJS1_ch02.pkl', 'gBR_sBM_cAll_d05_mBR5_ch10.pkl', 'gBR_sBM_cAll_d04_mBR3_ch10.pkl', 'gBR_sFM_cAll_d05_mBR4_ch11.pkl', 'gLH_sMM_cAll_d18_mLH2_ch08.pkl', 'gJS_sFM_cAll_d01_mJS3_ch04.pkl', 'gWA_sMM_cAll_d27_mWA2_ch08.pkl', 'gBR_sBM_cAll_d06_mBR4_ch08.pkl', 'gJS_sFM_cAll_d01_mJS1_ch07.pkl', 'gJB_sFM_cAll_d07_mJB2_ch03.pkl', 'gKR_sMM_cAll_d28_mKR1_ch02.pkl', 'gBR_sBM_cAll_d04_mBR1_ch10.pkl', 'gHO_sFM_cAll_d20_mHO5_ch13.pkl', 'gLO_sMM_cAll_d14_mLO1_ch04.pkl', 'gJB_sFM_cAll_d09_mJB1_ch16.pkl', 'gMH_sMM_cAll_d22_mMH2_ch03.pkl', 'gBR_sFM_cAll_d04_mBR3_ch04.pkl', 'gBR_sFM_cAll_d05_mBR5_ch14.pkl', 'gPO_sMM_cAll_d12_mPO4_ch10.pkl', 'gHO_sMM_cAll_d19_mHO0_ch01.pkl', 'gWA_sMM_cAll_d25_mWA1_ch01.pkl', 'gKR_sFM_cAll_d28_mKR5_ch06.pkl', 'gKR_sMM_cAll_d29_mKR1_ch05.pkl', 'gKR_sFM_cAll_d28_mKR3_ch07.pkl', 'gKR_sMM_cAll_d28_mKR0_ch01.pkl', 'gMH_sMM_cAll_d22_mMH0_ch01.pkl', 'gHO_sMM_cAll_d21_mHO2_ch07.pkl', 'gPO_sMM_cAll_d11_mPO5_ch06.pkl', 'gJS_sFM_cAll_d02_mJS3_ch04.pkl', 'gBR_sBM_cAll_d05_mBR4_ch10.pkl', 'gBR_sMM_cAll_d05_mBR4_ch06.pkl', 'gHO_sMM_cAll_d21_mHO4_ch09.pkl', 'gHO_sFM_cAll_d20_mHO3_ch14.pkl', 'gKR_sMM_cAll_d30_mKR0_ch07.pkl', 'gWA_sMM_cAll_d27_mWA5_ch10.pkl', 'gJS_sMM_cAll_d01_mJS1_ch02.pkl', 'gBR_sBM_cAll_d04_mBR2_ch08.pkl', 'gJS_sFM_cAll_d02_mJS0_ch08.pkl', 'gJS_sMM_cAll_d01_mJS0_ch01.pkl', 'gWA_sMM_cAll_d27_mWA3_ch09.pkl', 'gLH_sMM_cAll_d16_mLH2_ch02.pkl', 'gPO_sMM_cAll_d10_mPO2_ch01.pkl', 'gJB_sFM_cAll_d09_mJB2_ch17.pkl', 'gBR_sBM_cAll_d04_mBR1_ch08.pkl', 'gLO_sMM_cAll_d15_mLO1_ch08.pkl', 'gJS_sMM_cAll_d03_mJS3_ch03.pkl', 'gWA_sMM_cAll_d26_mWA2_ch04.pkl', 'gBR_sFM_cAll_d04_mBR2_ch03.pkl', 'gBR_sBM_cAll_d05_mBR5_ch07.pkl', 'gJB_sMM_cAll_d08_mJB3_ch05.pkl', 'gJB_sMM_cAll_d09_mJB3_ch09.pkl', 'gHO_sMM_cAll_d21_mHO3_ch08.pkl', 'gBR_sBM_cAll_d05_mBR4_ch08.pkl', 'gMH_sMM_cAll_d24_mMH0_ch07.pkl', 'gBR_sFM_cAll_d05_mBR2_ch09.pkl', 'gJB_sMM_cAll_d08_mJB1_ch04.pkl', 'gJB_sMM_cAll_d08_mJB4_ch06.pkl', 'gKR_sMM_cAll_d30_mKR3_ch10.pkl']

f_path = "./aist_dataset/aist_annotation/keypoints2d"	

for idx, filename in enumerate(tqdm(Skipped_files)):
        
    file_path = os.path.join(f_path, filename)
    with open(file_path, 'rb') as file:
            motion_data = pickle.load(file)

    markerA_x = motion_data["keypoints2d"][0, :, markerA_id, 0]     # array (n,)
    markerA_y = motion_data["keypoints2d"][0, :, markerA_id, 1]      # array (n,)
    
    print(len(markerA_x))
    break



In [ ]:
result = {
    "filename": [],
    "dance_genre": [],
    "situation": [],
    "camera_id": [],
    "dancer_id": [],
    "music_id": [],
    "choreo_id": [],
    "music_tempo": [],

    "X_a": [],
    "Y_a": [],

    "bpm_mode": [],
    "bpm_median": [],
    "mode_x": [],
    "mode_y": [],

    "median_x": [],
    "median_y": [],

}

json_filename = "music_id_tempo.json"
with open(json_filename, "r") as file:
    aist_tempo = json.load(file)

f_path = "./aist_dataset/aist_annotation/keypoints2d"
aist_filelist = os.listdir(f_path)

mocap_fps = 60
a = 70; b =145
tempi_range = np.arange(a,b,1)   # good: 70,145 

marker_dict = {9: "left_wrist", 10: "right_wrist", 
               15: "left_ankle", 16: "right_ankle", 
               11: "left_hip",12: "right_hip"}


sub_dir = "foot"
mode = "zero_bi"
markerA_id = 16          # 9: "left_wrist", 10: "right_wrist", 15: "left_ankle", 16: "right_ankle"
csv_filename = f"./results/aist_pos1s/z-score/tempo_{a}_{b}/{sub_dir}/{marker_dict[markerA_id]}_{mode}_{a}_{b}.csv"        

for idx, filename in enumerate(tqdm(aist_filelist)):
    
    file_path = os.path.join(f_path, filename)
    file_info = filename.split("_")
    dance_genre = file_info[0] 
    situation = file_info[1] 
    camera_id = file_info[2] 
    dancer_id = file_info[3]
    music_id = file_info[4]
    choreo_id = file_info[5].strip(".pkl")
    
    with open(file_path, 'rb') as file:
        motion_data = pickle.load(file)

    markerA_x = motion_data["keypoints2d"][0, :, markerA_id, 0]     # array (n,)
    markerA_y = motion_data["keypoints2d"][0, :, markerA_id, 1]      # array (n,)
    
    

    if np.all((markerA_x == 0) & (markerA_y == 0)) or np.any(np.isnan(markerA_x) | np.isnan(markerA_y)):
        continue
    
    markerA_x = detrend_signal_array(markerA_x.reshape(-1, 1), cutoff= 1, fs=60)
    markerA_y = detrend_signal_array(markerA_y.reshape(-1, 1), cutoff= 1, fs=60)
    markerA_pos = np.concatenate((markerA_x, markerA_y), axis=1)  # size (n,2)
    
    duration = int(len(markerA_x)/60)
    w_sec = int(duration)
    h_sec = int(w_sec/4)
    window_size = int(mocap_fps*w_sec)
    hop_size = int(mocap_fps*h_sec)
    
    bpm_axes = []
    for ax in range(2):
        # min max -1 to 1
        # x_min, x_max = np.min(markerA_pos[:, ax]), np.max(markerA_pos[:, ax])
        # markerA_pos_norm = (
        #     2 * (markerA_pos[:, ax] - x_min) / (x_max - x_min) - 1
        #     if x_max != x_min  # Avoid division by zero
        #     else np.zeros_like(markerA_pos[:, ax])
        # )
        
        # z-score
        mean_x = np.mean(markerA_pos[:, ax])
        std_x = np.std(markerA_pos[:, ax])
        
        markerA_pos_norm = (
        (markerA_pos[:, ax] - mean_x) / std_x if std_x != 0 else np.zeros_like(markerA_pos[:, ax])
        )

        markerA_ax = markerA_pos_norm.reshape(-1,1)
              
        tempo_json_one_sensor = main_one_sensor_peraxis(markerA_ax,
                                                        mocap_fps, window_size, hop_size, tempi_range, 
                                                        T_filter=0.25, smooth_wlen= 10, pk_order = 15 ,mode= mode)

        
        sensor_onsets = tempo_json_one_sensor["sensor_onsets"]
        tempo_data_maxmethod = tempo_json_one_sensor["tempo_data_maxmethod"]
        bpmA_arr = tempo_data_maxmethod["bpm_arr"]      # bpm per window
        tempo_A = np.round(np.average(bpmA_arr), 2)
        bpm_axes.append(bpmA_arr)
        
        mode_ax = stats.mode(bpmA_arr.flatten())[0]
        median_ax = np.median(bpmA_arr.flatten())

        if ax == 0:
            result["filename"].append(filename.strip(".pkl"))
            result["dance_genre"].append(dance_genre)
            result["situation"].append(situation)
            result["camera_id"].append(camera_id)
            result["dancer_id"].append(dancer_id)
            result["music_id"].append(music_id)
            result["choreo_id"].append(choreo_id)
            result["music_tempo"].append(aist_tempo[music_id])
            
            result["mode_x"].append(mode_ax)
            result["median_x"].append(median_ax)
            result["X_a"].append(tempo_A)
            
        elif ax == 1:
            result["mode_y"].append(mode_ax)
            result["median_y"].append(median_ax)
            result["Y_a"].append(tempo_A)

    bpm_axes_arr = np.column_stack(bpm_axes)    # n by 3 array
    bpm_mode = stats.mode(bpm_axes_arr.flatten())[0]
    bpm_median = np.median(bpm_axes_arr.flatten())
    result["bpm_mode"].append(bpm_mode)
    result["bpm_median"].append(bpm_median)    
    
    
# results_df = pd.DataFrame(result)
# results_df.to_csv(csv_filename, index=False)
# print(f"Results saved to {csv_filename}")

In [ ]:
print(markerA_ax.shape)
print(sensor_onsets.shape)
# print()

In [ ]:
# Extract x and y positions for the given keypoint
markerA_x = motion_data["keypoints2d"][0, :, 10, 0]
markerA_y = motion_data["keypoints2d"][0, :, 10, 1]

# Normalize x and y to [0,1] range
x_min, x_max = np.min(markerA_x), np.max(markerA_x)
y_min, y_max = np.min(markerA_y), np.max(markerA_y)

x_norm = (markerA_x - x_min) / (x_max - x_min) if x_max != x_min else np.zeros_like(markerA_x)
y_norm = (markerA_y - y_min) / (y_max - y_min) if y_max != y_min else np.zeros_like(markerA_y)

# Smooth the normalized position data
xs = smooth_velocity(x_norm.reshape(-1, 1), abs="no", window_length=10, polyorder=0)
ys = smooth_velocity(y_norm.reshape(-1, 1), abs="no", window_length=10, polyorder=0)


# Plot position and velocity
fig, axs = plt.subplots(1, 2, figsize=(10, 4), dpi=100)
fig.suptitle('Position and Velocity')

# Position plots
axs[0].plot(xs, label="x position")
axs[1].plot(ys, label="y position")
axs[0].set_title("X Position")
axs[1].set_title("Y Position")

# Formatting
for ax in axs.flat:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
from aist_pos1s_script import load_from_json
a = 70
b = 145

savepath = f"./results/aist_pos1s/z-score/tempo_{a}_{b}/tempo_data/json/ax0"

right_ankle_json = load_from_json(savepath, "right_ankle_gLO_sBM_cAll_d15_mLO5_ch06_axis0.json")

In [ ]:
right_ankle_json.keys()
right_ankle_json['tempo_data_maxmethod'].keys()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(right_ankle_json['sensor_abs'])
plt.show()